# Previsão de Churn em Clientes SaaS

1. Pré-processamento e tratamento de dados
2. Feature Engineering detalhado
3. Seleção de variáveis (multicolinearidade, SelectKBest, RFE)
4. Divisão treino/teste e validação cruzada
5. Treinamento comparativo de modelos: Regressão Logística, Random Forest, XGBoost e SVM
6. Avaliação de performance (AUC, F1, precisão, recall)
7. Verificação de overfitting/underfitting
8. Explainable AI (SHAP) para o modelo final



## 1. Importando bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, RocCurveDisplay
from statsmodels.stats.outliers_influence import variance_inflation_factor
import shap
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 2. Carregando e visualizando os dados

In [ ]:
# Carrega a base de dados
df = pd.read_csv('/mnt/data/base_churn_saas.csv')
print('Dimensões da base:', df.shape)
df.head()

## 3. Pré-processamento e tratamento inicial

In [ ]:
# 3.1 Verificando valores faltantes
print(df.isnull().sum())

**Insight**: Não há valores faltantes, portanto não será necessário imputação.

In [ ]:
# 3.2 Remover colunas irrelevantes
df.drop(columns=['id_cliente', 'codigo_cliente', 'constante'], inplace=True)
print('Novas dimensões:', df.shape)

Removemos identificadores e a coluna constante, pois não agregam nenhuma informação preditiva.

In [ ]:
# 3.3 Convertendo variáveis categóricas para o tipo 'category'
categorical_cols = ['plano', 'regiao', 'navegador_usado']
for col in categorical_cols:
    df[col] = df[col].astype('category')
df.dtypes

Convertidas variáveis categóricas para o tipo adequado para facilitar codificação posterior.

## 4. Feature Engineering

Vamos criar novas variáveis com base em conhecimento de negócio:
- `uso_total`: combinação de frequência de uso e tempo total de login
- `dias_sem_login`: cópia de `ultimo_login_dias`
- `suporte_freq`: relação entre chamados abertos e tempo de contrato
- `satisfacao_baixa`: flag se avaliação de satisfação < 3
- `acesso_noturno`: flag se `hora_ultimo_acesso` entre 0 e 6


In [ ]:
# 4.1 Criando features derivadas
df['uso_total'] = df['frequencia_uso_mensal'] * df['tempo_total_login_horas']
df['dias_sem_login'] = df['ultimo_login_dias']
df['suporte_freq'] = df['suporte_chamados_abertos'] / (df['tempo_de_contrato_meses'] + 1)
df['satisfacao_baixa'] = (df['avaliacao_satisfacao'] < 3).astype(int)
df['acesso_noturno'] = df['hora_ultimo_acesso'].apply(lambda x: 1 if x <= 6 else 0)
df[['uso_total','suporte_freq','satisfacao_baixa','acesso_noturno']].head()

Criamos variáveis que podem capturar comportamentos de risco de churn.

## 5. Codificação de variáveis categóricas e escalonamento

Vamos separar variáveis numéricas e categóricas para construir um `ColumnTransformer`.

In [ ]:
# 5.1 Definir X e y
X = df.drop('churn', axis=1)               # Cria X retirando a coluna 'churn' do DataFrame df; axis=1 indica remoção de coluna
y = df['churn']                            # Cria y contendo apenas a coluna 'churn' (variável-alvo)

# Identificar colunas numéricas (tipo inteiro ou float) e categóricas (tipo category)
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
                                           # select_dtypes seleciona colunas cujo dtype é int64 ou float64
                                           # .columns.tolist() converte o índice de colunas em lista de nomes

cat_cols = X.select_dtypes(include=['category']).columns.tolist()
                                           # select_dtypes seleciona colunas cujo dtype é category
                                           # .columns.tolist() converte em lista de nomes

print('Numéricas:', num_cols)              # Exibe a lista de nomes de colunas numéricas
print('Categóricas:', cat_cols)             # Exibe a lista de nomes de colunas categóricas


# 5.2 Construir ColumnTransformer: OneHotEncoder para categóricas e StandardScaler para numéricas
preprocessor = ColumnTransformer([
    ('onehot', OneHotEncoder(drop='first', sparse=False), cat_cols),
                                           # Cria um transformador chamado 'onehot' que aplica OneHotEncoder às colunas categóricas
                                           # drop='first' elimina uma coluna dummy para evitar multicolinearidade
                                           # sparse=False retorna array NumPy denso em vez de matriz esparsa
    ('scale', StandardScaler(), num_cols)    # Cria um transformador chamado 'scale' que aplica StandardScaler às colunas numéricas
])
                                           # ColumnTransformer combina esses dois transformadores:
                                           #   - OneHotEncoder nas colunas listadas em cat_cols
                                           #   - StandardScaler nas colunas listadas em num_cols
                                           # As demais colunas (se houvesse) seriam descartadas por default (não pass-through)


Nosso preprocessor fará codificação one-hot e escalonamento padrão.

## 6. Seleção de Variáveis

Primeiro, vamos verificar multicolinearidade com VIF. Para isso, precisamos do dataset codificado e padronizado.

In [ ]:
# 6.1 Gerar matriz codificada para calcular VIF
X_encoded = preprocessor.fit_transform(X)
feature_names = list(preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_cols)) + num_cols
X_vif = pd.DataFrame(X_encoded, columns=feature_names)
# Calcular VIF
vif_data = pd.DataFrame()
vif_data['feature'] = feature_names
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
vif_data.sort_values(by='VIF', ascending=False).head(10)

As variáveis com VIF > 5 indicam multicolinearidade alta. Avalie remover ou combinar.
Neste caso, `frequencia_uso_mensal` e `tempo_total_login_horas` apresentam leve correlação.


### 6.2 SelectKBest (Chi²)

In [ ]:
# Aplicar SelectKBest para selecionar as 10 variáveis mais relevantes
selector = SelectKBest(score_func=chi2, k=10)
X_best = selector.fit_transform(pd.DataFrame(X_encoded, columns=feature_names), y)
best_features = np.array(feature_names)[selector.get_support()]
print('Features selecionadas pelo SelectKBest:', best_features)

Selecionamos as 10 variáveis que melhor se relacionam com `churn` segundo teste Chi².

### 6.3 Recursive Feature Elimination (RFE) com Regressão Logística

In [ ]:
# Usar RFE para selecionar 10 variáveis com base em coeficientes de regressão logística
lr = LogisticRegression(max_iter=1000)
rfe = RFE(estimator=lr, n_features_to_select=10)
rfe.fit(pd.DataFrame(X_encoded, columns=feature_names), y)
rfe_features = np.array(feature_names)[rfe.get_support()]
print('Features selecionadas pelo RFE:', rfe_features)

Podemos comparar `SelectKBest` e `RFE` e optar pelo conjunto que gerar melhor performance.

## 7. Divisão entre Treino e Teste

In [ ]:
# Definimos o conjunto final de features (por exemplo, best_features do SelectKBest)
X_selected = pd.DataFrame(X_encoded, columns=feature_names)[best_features]

# Dividir em 80% treino e 20% teste, mantendo proporção de churn
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y)
print('Shape X_train:', X_train.shape)
print('Shape X_test:', X_test.shape)

Usamos `stratify=y` para manter a proporção de churn em treino e teste (20% de churn).

## 8. Treinamento e Avaliação de Modelos

Treinaremos quatro modelos:
- Regressão Logística
- Random Forest
- XGBoost
- SVM (com probabilidade)
Em seguida, compararemos as métricas (Acurácia, ROC AUC, Precision, Recall, F1-score).

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
}

results = []
for name, model in models.items():
    print(f'Treinando {name}...')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    report = classification_report(y_test, y_pred, output_dict=True)
    results.append({
        'Modelo': name,
        'Accuracy': acc,
        'ROC AUC': auc,
        'Precision': report['1']['precision'],
        'Recall': report['1']['recall'],
        'F1-score': report['1']['f1-score']
    })

# DataFrame de resultados
df_results = pd.DataFrame(results).sort_values(by='ROC AUC', ascending=False)
df_results

Os resultados acima permitem comparar qual modelo tem melhor desempenho para prever churn. Normalmente, priorizamos **ROC AUC** em problemas desbalanceados.

## 9. Verificação de Overfitting / Underfitting

In [ ]:
# Avaliar acurácia em treino x teste para o melhor modelo (ex.: XGBoost se for o melhor)
best_model_name = df_results.iloc[0]['Modelo']
best_model = models[best_model_name]
train_acc = accuracy_score(y_train, best_model.predict(X_train))
test_acc = accuracy_score(y_test, best_model.predict(X_test))
print(f'Best Model: {best_model_name}')
print(f'Train Accuracy: {train_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

# Plot curva ROC treino x teste
RocCurveDisplay.from_estimator(best_model, X_train, y_train, name='Train ROC')
RocCurveDisplay.from_estimator(best_model, X_test, y_test, name='Test ROC', ax=plt.gca())
plt.title(f'Curvas ROC - {best_model_name}')
plt.show()

Se a performance em treino for muito superior à de teste, há overfitting. Curvas ROC se sobrepondo indicam bom ajuste.

## 10. Explainable AI (SHAP)

Vamos usar SHAP para entender a contribuição de cada feature nas previsões do melhor modelo.

In [ ]:
# 10.1 Criar objeto explainer (apenas para modelos baseados em árvores, ex.: Random Forest ou XGBoost)
import shap

if best_model_name in ['Random Forest', 'XGBoost']:
    explainer = shap.TreeExplainer(best_model)
else:
    explainer = shap.Explainer(best_model, X_train)

# 10.2 Calcular valores SHAP para conjunto de teste
shap_values = explainer(X_test)

# 10.3 Plot summary (global)
shap.summary_plot(shap_values, X_test, plot_type='bar')

O gráfico de barras mostra a importância média de cada feature. As barras maiores indicam features mais influentes.

### 10.4 Explicação local (exemplo de um cliente)

In [ ]:
# Escolher um exemplo do conjunto de teste para explicar localmente
idx = 5  # índice arbitrário
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[idx], X_test.iloc[idx])

Esse `force_plot` mostra, para um cliente específico, quais features aumentaram ou diminuíram a probabilidade de churn.

## 11. Conclusão

Neste notebook, avançamos desde o pré-processamento até a construção e avaliação de múltiplos modelos, além de usar SHAP para explicar o comportamento do modelo final. Com esses passos, temos um pipeline completo para prever churn em clientes SaaS, com insights para ação e comunicação clara aos stakeholders.